In [1]:
import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime


ModuleNotFoundError: No module named 'azureml.train.automl'

In [26]:
ws = Workspace.from_config()
print(ws)
# choose a name for the run history container in the workspace
experiment_name = 'automl-stockforecasting'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

Workspace.create(name='stockforecast_ws', subscription_id='752827ae-b752-4c3c-a8ad-510c6eab7c6a', resource_group='stockforecast_rg')


/Users/jasonxiao/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  from ipykernel import kernelapp as app


,
Subscription ID,752827ae-b752-4c3c-a8ad-510c6eab7c6a
Workspace,stockforecast_ws
SKU,Basic
Resource Group,stockforecast_rg
Location,eastus2
Run History Name,automl-stockforecasting


In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "stock-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [13]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./MSFT.csv'], target_path = 'dataset/', overwrite = True,show_progress = True)

Uploading an estimated of 1 files
Uploading ./MSFT.csv
Uploaded ./MSFT.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_4ae14fbc43304544b76d76a1f230f782

In [14]:
target_column_name = 'close'
time_column_name = 'date'

In [22]:
dataset = Dataset.Tabular.from_delimited_files(path = [("workspaceblobstore", 'dataset/MSFT.csv')]).with_timestamp_columns(fine_grain_timestamp=time_column_name) 
dataset.take(5).to_pandas_dataframe().reset_index(drop=True)

UserErrorException: UserErrorException:
	Message: Invalid tuple for path. Please make sure the tuple consists of a datastore and a path/SQL query
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Invalid tuple for path. Please make sure the tuple consists of a datastore and a path/SQL query"
    }
}